In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
import random


## Getting glyco impact for all amino acid substitutions

In [2]:
gi = pd.read_csv('../data/glyco_protein_impact_within_6.csv')

In [3]:
gi_copy = gi.copy()

In [4]:
gi_copy['substitution'] = ['=>'.join([g.split('=>')[-1],g.split('=>')[0]])for g in gi_copy['substitution']]

In [5]:
gi = pd.concat([gi_copy,gi])

## Getting rank normalized functional impact scores across many different algorithms as well as the corresponding glycodistances of those mutations

In [6]:
df = pd.read_csv('../data/glycoproteins_distances_with_pathsoftware.txt',
                 sep='\t',
                 index_col=0)

/Users/d/work/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (7,8,19,27,34,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/d/work/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
df = df.dropna(subset=['glyco_distance'])

In [8]:
df.shape

(3549910, 38)

# Calculating correlation between differences betwen every pair of software functional prediction score and glycodistance of that mutation 

In [9]:
software = df.columns[df.columns.str.contains('rank')].to_list()

In [25]:
corr_dct = {}
random_dct = {}

for s1 in software:
    corr_dct[s1] = {}
    random_dct[s1] = {}
    
    for s2 in software: 
        pair_soft = df[(df[s1] != '.') & (df[s2] != '.')]
        pair_soft = pair_soft[[s1,s2,'glyco_distance']]
        pair_soft = pair_soft.dropna(subset=[s1,s2])
        
        # Computing correlation between software 
        # I am using .iloc to get the columns due to case of when s1 == s2
        soft_corr = scipy.stats.spearmanr(pair_soft.iloc[:,0].to_list(),pair_soft.iloc[:,1].to_list())
        corr_dct[s1][s2] = [soft_corr[0],soft_corr[1]]
        
        # Getting difference between predictions
        diff = pair_soft.iloc[:,1].astype(float) - pair_soft.iloc[:,0].astype(float)
        
        # Computing correlation between the difference in software and the glycodistance 
        diff_corr = scipy.stats.spearmanr(diff,pair_soft['glyco_distance'].to_list())
        corr_dct[s1][s2].extend([diff_corr[0],diff_corr[1]])
        
        # Shuffling glycosylation distances
        random_glyco = random.sample(pair_soft['glyco_distance'].to_list(),k=pair_soft.shape[0])
        
        # Computing shuffled correlation b/w difference in software and glycodistance
        random_results = scipy.stats.spearmanr(diff,random_glyco)
        
        random_dct[s1][s2] = [random_results[0],random_results[1]]

/Users/d/work/conda/lib/python3.7/site-packages/scipy/stats/stats.py:248: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)
/Users/d/work/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/d/work/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/d/work/conda/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/d/work/conda/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/d/work/conda/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py

## Saving dataframes to files

### Real data

In [29]:
correlation_dcts = {'corr_rho':{},
                    'corr_p':{},
                    'diff_rho':{},
                    'diff_p':{}}

assign = [('corr_rho',0),
        ('corr_p',1),
        ('diff_rho',2),
        ('diff_p',3)]

for a in assign:
    for s1 in corr_dct.keys():
        correlation_dcts[a[0]][s1] = {}
        correlation_dcts[a[0]][s1] = {s2:corr_dct[s1][s2][a[1]] for s2 in corr_dct[s1].keys()}

for df_type in correlation_dcts.keys():
    df = pd.DataFrame.from_dict(correlation_dcts[df_type])
    df.to_csv('../data/{}_df.csv'.format(df_type))

### Null data

In [48]:
correlation_dcts = {'rand_rho':{},
                    'rand_p':{}}

assign = [('rand_rho',0),
        ('rand_p',1)]

for a in assign:
    for s1 in corr_dct.keys():
        correlation_dcts[a[0]][s1] = {}
        correlation_dcts[a[0]][s1] = {s2:random_dct[s1][s2][a[1]] for s2 in random_dct[s1].keys()}

for df_type in correlation_dcts.keys():
    df = pd.DataFrame.from_dict(correlation_dcts[df_type])
    df.to_csv('../data/{}_df.csv'.format(df_type))